In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
X = train_df.iloc[:, 1:]
y = train_df.iloc[:, 0]

In [3]:
X = np.array(X)
y = np.array(y)

In [4]:
X = X.astype('float32') / 255

In [5]:
from sklearn.model_selection import train_test_split
X_dev, X_val, y_dev, y_val = train_test_split(X, y, test_size=0.03, shuffle=True, random_state=2022)

X_dev = X_dev.reshape(X_dev.shape[0], 28, 28, 1)
X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)

from tensorflow.keras.utils import to_categorical
y_dev = pd.get_dummies(y_dev).values
y_val = pd.get_dummies(y_val).values

In [6]:
from tensorflow.keras import  Model, Input
from tensorflow.keras.layers  import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten

input_tensor = Input(shape=(64,))
x = Dense(32, activation='relu')(input_tensor)
x = Dense(32, activation='relu')(x)
output_tensor = Dense(10, activation='softmax')(x)
func_model = Model(input_tensor, output_tensor)

In [7]:
# function for creating a naive inception block
def inception_block(layer_in, f1, f2, f3):
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
    conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
    conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out

In [8]:
from tensorflow.keras.layers import concatenate

inp = Input(shape=(28, 28, 1))
x = inception_block(inp, 16, 32, 16)
x = Flatten()(x)
out = Dense(10, activation='softmax')(x)
inception_model = Model(inp, out)

In [9]:
from tensorflow.keras.optimizers import Adam
inception_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=5e-4))
inception_model.fit(X_dev, y_dev, epochs=50, batch_size=64)

Epoch 1/50
637/637 [==============================] - 14s 5ms/step - loss: 0.2476 - accuracy: 0.9309
Epoch 2/50
637/637 [==============================] - 3s 5ms/step - loss: 0.0853 - accuracy: 0.9758
Epoch 3/50
637/637 [==============================] - 3s 5ms/step - loss: 0.0575 - accuracy: 0.9836
Epoch 4/50
637/637 [==============================] - 3s 4ms/step - loss: 0.0429 - accuracy: 0.9869
Epoch 5/50
637/637 [==============================] - 3s 4ms/step - loss: 0.0343 - accuracy: 0.9893
Epoch 6/50
637/637 [==============================] - 3s 5ms/step - loss: 0.0266 - accuracy: 0.9922
Epoch 7/50
637/637 [==============================] - 3s 4ms/step - loss: 0.0229 - accuracy: 0.9931
Epoch 8/50
637/637 [==============================] - 3s 4ms/step - loss: 0.0176 - accuracy: 0.9950
Epoch 9/50
637/637 [==============================] - 3s 4ms/step - loss: 0.0146 - accuracy: 0.9960
Epoch 10/50
637/637 [==============================] - 3s 5ms/step - loss: 0.0121 - accuracy: 0.996

In [10]:
performance = inception_model.evaluate(X_val, y_val, batch_size=32)
dict(zip(inception_model.metrics_names, performance))

40/40 [==============================] - 0s 4ms/step - loss: 0.1165 - accuracy: 0.9825


{'loss': 0.11645561456680298, 'accuracy': 0.9825396537780762}

In [11]:
X_test = np.array(pd.read_csv('/kaggle/input/digit-recognizer/test.csv'))/255.0
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

y_test = inception_model.predict(X_test)
y_test

875/875 [==============================] - 2s 2ms/step


array([[1.7225357e-28, 0.0000000e+00, 1.0000000e+00, ..., 1.2176154e-25,
        3.4647885e-25, 2.8167192e-21],
       [1.0000000e+00, 1.6443555e-38, 9.8421382e-18, ..., 1.3002373e-22,
        3.5811678e-21, 2.9890883e-27],
       [7.3335733e-21, 1.3531811e-18, 1.8109848e-10, ..., 2.7414666e-12,
        6.9564270e-09, 1.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 1.1108740e-25, ..., 6.4760843e-27,
        8.2294269e-21, 2.0810696e-18],
       [3.5477221e-23, 1.9405484e-28, 3.6646314e-21, ..., 4.0099573e-15,
        2.8554730e-09, 9.9999046e-01],
       [1.5349727e-30, 0.0000000e+00, 1.0000000e+00, ..., 2.0177271e-25,
        1.9514531e-17, 1.2378018e-11]], dtype=float32)

In [12]:
y = np.argmax(y_test, axis=1)

In [13]:
test_sub = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
test_sub["Label"] = y
test_sub.to_csv("/kaggle/working/submission.csv", index=False)